In [1]:
!pip install findspark

In [2]:
import findspark

In [3]:
findspark.init("C:\spark")

In [4]:
from pyspark import SparkContext

In [5]:
#Spark Uygulamasının Başlatılması

In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [12]:
spark = SparkSession.builder \
    .master("local") \
    .appName("pyspark_giriş") \
    .getOrCreate()

sc = spark.sparkContext

In [13]:
sc

<SparkContext master=local appName=pyspark_giriş>

In [14]:
sc.version

'3.3.2'

In [16]:
sc.appName

'pyspark_giriş'

In [ ]:
#sc.stop() Başlanılan uygulamayı mutlaka kapatmamız gerekir.

In [17]:
# Temel DataFrame İşlemleri

In [19]:
spark_df = spark.read.csv("./churn.csv", header = True, inferSchema = True)

In [20]:
spark_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Churn: integer (nullable = true)



In [21]:
type(spark_df)

pyspark.sql.dataframe.DataFrame

In [22]:
spark_df.cache()

DataFrame[_c0: int, Names: string, Age: double, Total_Purchase: double, Account_Manager: int, Years: double, Num_Sites: double, Churn: int]

In [24]:
spark_df.head() #İlk gözlem birimine ulaştık

Row(_c0=0, Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Churn=1)

In [27]:
spark_df.show(3, truncate = True) # Yukarıda bulunan head() değerini daha yapısal forma dönüştürdük.

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 3 rows



In [28]:
spark_df.count() #Gözlem sayısına ulaşmak için.

900

In [29]:
spark_df.columns #Değişken ve bu değişkenlerin isimlerine ulaşmak istersek bunu kullanacağız.

['_c0',
 'Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Churn']

In [32]:
spark_df.describe().show() #Özet istatistik istersek bunu kullanacağız.

+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|summary|               _c0|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|              Churn|
+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|  count|               900|          900|              900|              900|               900|              900|               900|                900|
|   mean|             449.5|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|259.95191863111916|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|                 0|   Aaron King|             22.0|          

In [34]:
spark_df.describe("Age").show() #Bir gözlemein değerlerine ulaşmak için bunu kullanıyoruz.

+-------+-----------------+
|summary|              Age|
+-------+-----------------+
|  count|              900|
|   mean|41.81666666666667|
| stddev|6.127560416916251|
|    min|             22.0|
|    max|             65.0|
+-------+-----------------+



In [35]:
spark_df.select("Age", "Names").show() #Değişken içinden istediğimi getirmek istiyorum

+----+-------------------+
| Age|              Names|
+----+-------------------+
|42.0|   Cameron Williams|
|41.0|      Kevin Mueller|
|38.0|        Eric Lozano|
|42.0|      Phillip White|
|37.0|     Cynthia Norton|
|48.0|   Jessica Williams|
|44.0|        Eric Butler|
|32.0|      Zachary Walsh|
|43.0|        Ashlee Carr|
|40.0|     Jennifer Lynch|
|30.0|       Paula Harris|
|45.0|     Bruce Phillips|
|45.0|       Craig Garner|
|40.0|       Nicole Olson|
|41.0|     Harold Griffin|
|38.0|       James Wright|
|45.0|      Doris Wilkins|
|43.0|Katherine Carpenter|
|53.0|     Lindsay Martin|
|46.0|        Kathy Curry|
+----+-------------------+
only showing top 20 rows



In [36]:
spark_df.filter(spark_df.Age > 40).count() #Gözlemler ile ilgili seçme işlemini yapalım

524

In [37]:
spark_df.groupby("churn").count().show() #Gruplama işlemi yapalım.

+-----+-----+
|churn|count|
+-----+-----+
|    1|  150|
|    0|  750|
+-----+-----+



1 olanlar markayı terk edenler, 0 olanlaar terk etmeyenler.

In [ ]:
#Aggregion İşlemi Yapalım

In [38]:
spark_df.groupby("churn").agg({"Age":"mean"}).show()

+-----+-----------------+
|churn|         avg(Age)|
+-----+-----------------+
|    1|42.99333333333333|
|    0|41.58133333333333|
+-----+-----------------+



In [62]:
#Spark'da SQL İşlemleri

In [40]:
spark_df.createOrReplaceTempView("tbl_df")

In [42]:
spark.sql("show databases").show() #default olarak dv var onu gösterdi bize şuan.

+---------+
|namespace|
+---------+
|  default|
+---------+



In [55]:
spark.sql("show tables").show() #sadece bir tablo gözüktü o da bizim oluşturduğumuz tablo. 

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |   tbl_df|       true|
+---------+---------+-----------+



In [77]:
spark.sql("select Age from tbl_df order by age desc ").show(10)

+----+
| Age|
+----+
|65.0|
|60.0|
|58.0|
|58.0|
|56.0|
|56.0|
|56.0|
|56.0|
|56.0|
|55.0|
+----+
only showing top 10 rows



In [61]:
spark.sql("select Churn, mean(Age) from tbl_df group by Churn").show()

+-----+-----------------+
|Churn|        mean(Age)|
+-----+-----------------+
|    1|42.99333333333333|
|    0|41.58133333333333|
+-----+-----------------+



In [63]:
#Büyük Veri'de Görselleştrme İşlemleri

In [76]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.barplot(x = "Churn", y = spark_df.Churn.index = spark_df)

#Hata verdi çünkü pandas dataframe arka planda çalışıyor biz spark kullandık. Bunu dönüştürmemiz gerekiyor.

sdf = spark_df.toPandas()

Yukarıda çevirme işlemini tamamladık. İşlemi tekrar yapıyorum.

sns.barplot(x = "Churn", y = sdf.Churn.index = spark_df)

a = spark_df.groupby("Churn").count().toPandas()
a

type(a) #Denildiğinde  pandas dataframe dönüştürüldü.

VERİ GÖRSELLEŞTİRME DE ÖNCE "TODATAFRAME" KULLANARAK DÖNÜŞTÜRÜLME İŞLEMİ YAPILDIKTAN SONRA GÖRSELLEŞTİRM İŞLEMİ YAPILIR.

In [107]:
#BÜYÜK VERİ'DE MAKİNE ÖĞRENMESİ KONUSU

In [108]:
#Müşteri Terk Projesi

In [109]:
spark_df = spark.read.csv("./churn.csv", header = True, inferSchema = True)

In [110]:
spark_df.show(5)

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [111]:
#Yukarıda df e görüldüğü üzre değişkenlerin isimleri kuralsız bunu düzeltelim.

In [114]:
spark_df = spark_df.toDF(*[c.lower() for c in spark_df.columns])

In [115]:
spark_df.show(5)

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           names| age|total_purchase|account_manager|years|num_sites|churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
+---+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [116]:
#İkinci rahatsız eden şey df de bulunan c değişkeni

In [119]:
spark_df = spark_df.withColumnRenamed("_c0", "index") #ismi eğişmiş oldu.

In [118]:
spark_df.show(5)

+-----+----------------+----+--------------+---------------+-----+---------+-----+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|
+-----+----------------+----+--------------+---------------+-----+---------+-----+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|    3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|    4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
+-----+----------------+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [121]:
spark_df.count() #Gözlem değeri

900

In [123]:
len(spark_df.columns) #8 tane değişken var.

8

In [124]:
spark_df.columns #Değişken isimlerine ise bu şekilde erişebiliyoruz.

['index',
 'names',
 'age',
 'total_purchase',
 'account_manager',
 'years',
 'num_sites',
 'churn']

In [125]:
spark_df.describe().show() #Betimsel istatistik

+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|summary|             index|        names|              age|   total_purchase|   account_manager|            years|         num_sites|              churn|
+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|  count|               900|          900|              900|              900|               900|              900|               900|                900|
|   mean|             449.5|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|259.95191863111916|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|                 0|   Aaron King|             22.0|          

In [126]:
#Yukarıda istatistiksel çıktılar tam olarak güzel bir şekilde gözükmüyor. Bu neden ile aşağıda ki gibi yapacağız.

spark_df.select("age", "total_purchase", "account_manager", "years", "num_sites", "churn").describe().toPandas().Transpose()

In [ ]:
#Veri ön işleme yapalım df de. Örneğin eksik gözlemlerden kurtulmak istiyoruz.

In [128]:
spark_df = spark_df.dropna()

In [129]:
#Yeni bir değişken oluşturalım.

In [130]:
spark_df = spark_df.withColumn("age_kare", spark_df.age**2)

In [135]:
spark_df.show(3)

+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|age_kare|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|  1764.0|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|  1681.0|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|  1444.0|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+
only showing top 3 rows



In [136]:
#Bağımlı Değişken Belirtme

Elimizde bir string değişken varmış gibi kabul edip bir kategorik değişken bunun bir bağımlı değişken olduğunu göstermiş olacağız.

In [138]:
from pyspark.ml.feature import StringIndexer

In [142]:
stringIndexer = StringIndexer(inputCol = "churn", outputCol = "label")
mod = stringIndexer.fit(spark_df)
indexed = mod.transform(spark_df)
spark_df = indexed.withColumn("label", indexed["label"].cast("integer"))

In [143]:
spark_df.show(3)

+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|age_kare|label|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|  1764.0|    1|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|  1681.0|    1|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|  1444.0|    1|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+
only showing top 3 rows



In [145]:
#Bağımsız Değişkenler

In [146]:
from pyspark.ml.feature import VectorAssembler

In [147]:
spark_df.columns

['index',
 'names',
 'age',
 'total_purchase',
 'account_manager',
 'years',
 'num_sites',
 'churn',
 'age_kare',
 'label']

In [149]:
bagimsiz_degiskenler = ["age", "total_purchase", "account_manager", "years", "num_sites"]

In [150]:
vectorAssembler = VectorAssembler(inputCols = bagimsiz_degiskenler, outputCol = "features")
va_df = vectorAssembler.transform(spark_df)

In [154]:
va_df.show(3)

+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+--------------------+
|index|           names| age|total_purchase|account_manager|years|num_sites|churn|age_kare|label|            features|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+--------------------+
|    0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|  1764.0|    1|[42.0,11066.8,0.0...|
|    1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|  1681.0|    1|[41.0,11916.22,0....|
|    2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|  1444.0|    1|[38.0,12884.75,0....|
+-----+----------------+----+--------------+---------------+-----+---------+-----+--------+-----+--------------------+
only showing top 3 rows



In [155]:
#Yukarıda hem bağımlı hem de bağımsız değişkenlerin tamamı var , biz burada bunların içerisiden seçme işlemine sokacağız 

In [156]:
final_df = va_df.select(["features", "label"])

In [157]:
final_df.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
+--------------------+-----+
only showing top 3 rows



Biz burada bağımlı değişken ve bağımsız değişkeni bir arada göstermiş olduk.

In [159]:
#Test - Train

In [160]:
splits = final_df.randomSplit([0.70, 0.30])
train_df = splits[0]
test_df = splits[1]

In [161]:
train_df

DataFrame[features: vector, label: int]

In [162]:
test_df

DataFrame[features: vector, label: int]

YUKARIDA VERİYİ MODELLEME İŞLEMİNE HAZIR HALE GETİRDİK.

In [ ]:
#GBM ile MÜŞTERİ TERK MODELLEMESİ

Amacımız bağımlı değişken ile bağımsız değişken arsındaki ilişkiyi modelleyip müşterilerimizin bizi terk edip etmeyeceğini tahminlemeye çalışmak.

In [164]:
from pyspark.ml.classification import GBTClassifier

In [166]:
# "GBTClassifier" bu fonksiyon ile modelleme işlemi yapacağız ve işlem bittiğinde elimize yeni müşteriler ve bilgileri gelecek bu müşterilerin bilgilerine göre bizi terk edip etmeyecğini tahmin etmeye çalışacağız.
gbm = GBTClassifier(maxIter = 10, featuresCol = "features", labelCol = "label")

In [167]:
gbm_model = gbm.fit(train_df)

In [168]:
sc

<SparkContext master=local appName=pyspark_giriş>

In [169]:
#Modeli kurduk şimdi model üzerinden tahminleme işlemi yapalım şimi aşağıda

In [170]:
y_pred = gbm_model.transform(test_df) # test_df ne ilişkin tahmin deilen y değerlerini kayıt etmiş olduk.

In [173]:
y_pred

DataFrame[features: vector, label: int, rawPrediction: vector, probability: vector, prediction: double]

In [172]:
ac = y_pred.select("label", "prediction") #y_pred nesnesinin içerisinde label ve prediction da içeren çıktılar var. 

In [175]:
ac.filter(ac.label == ac.prediction).count()/ac.count() #Gerçek değerler ile tahmin edilen değerleri karşılaştıracak ve true, fale değerleri dönecek. True olanları sayıcak ve bütün ac değerine bölecek.

0.8874172185430463

#MODEL TUNING